In [1]:
# Libraries
import pandas as pd
import numpy as np
import re
import xlsxwriter
import unidecode

In [10]:
def get_update(excel_file, sheet):
    
    # Open files
    transacties = pd.read_excel(excel_file, sheet_name= sheet)
    sjaarzen = pd.read_excel('Sjaarzen geslacht + naam + mail + nr.xlsx', 'Sheet1')
    
    # Bewerk namen sjaarzen
    voornaam = []
    achternaam = []
    
    for vnaam in sjaarzen['Naam']:
        x = unidecode.unidecode(str(vnaam))
        y = re.sub('-', '', str(x))
        z = re.sub(r'\s', '', y)
        voornaam.append(z)
    
    for anaam in sjaarzen['Achternaam']:
        x = unidecode.unidecode(str(anaam))
        y = re.sub('-', '', str(x))
        z = re.sub(r'\s', '', y)
        achternaam.append(z)
    
    # Vervang namen
    sjaarzen['Naam'] = voornaam
    sjaarzen['Achternaam'] = achternaam
    
    # Empty lists
    tikkies = []
    tikkieID = []
    naam = []
    
    # Filter alleen de tikkies uit alle transacties
    for omschrijving in transacties['Omschrijving']:
        x = re.findall(r'^.*Tikkie.*$', str(omschrijving))
        if x != []:
            tikkies.append(x[0])
            
    # Creeer nieuw DF
    collecte = transacties[transacties['Omschrijving'].isin(tikkies)]

    # Filter voor Tikkie ID en Naam
    for ID in tikkies:
        x = re.search(r'Tikkie\sID\s\d+', ID)
        tikkieID.append(x[0])
        y = ID.split('/')
        if y[-1] != []:
            a = re.sub(r'\s', '', y[-1])
            naam.append(a)
    
    # Maak nieuw DF
    totaal_collecte = pd.DataFrame({'Naam': naam,
                          'TikkieID': tikkieID,
                          'Opgehaald': collecte['Transactiebedrag']
                         }).groupby(['Naam']).sum()

    totaal_collecte = totaal_collecte.sort_values(by = ['Opgehaald'], ascending = False)
            
    # Format DF      
    sjaarzen['CombiNaam'] = sjaarzen['Naam'] + sjaarzen['Achternaam']
    sjaarzen.drop('Unnamed: 5', axis = 1, inplace = True)
        
    # Merge DF and format
    sjaarzen2 = sjaarzen.merge(totaal_collecte, how = 'left',
                               right_on = 'Naam', left_on = 'CombiNaam')
    sjaarzen2 = sjaarzen2.drop('CombiNaam', axis = 1)
    sjaarzen2 = sjaarzen2.drop(index = [379, 380])
    sjaarzen2 = sjaarzen2.fillna(value = 0)    
    
    # Save to excel file
    writer = pd.ExcelWriter('Update.xlsx', engine='xlsxwriter')
    sjaarzen2.to_excel(writer, sheet_name = 'Sheet1')
    writer.save()

    return sjaarzen2

In [11]:
update = get_update('updates/laatste versie.xls', 'Sheet0')
update['Opgehaald'].sum()

69104.38